# Experiment 12 timesfm

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 95.5 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 357MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [3]:
!pip install timesfm torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
import time

# Core libraries
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# TimesFM libraries
try:
    import torch
    import timesfm
    TIMESFM_AVAILABLE = True
    print("✅ TimesFM and PyTorch are available")
    print(f"   🔥 PyTorch version: {torch.__version__}")
    print(f"   🧠 CUDA available: {torch.cuda.is_available()}")
except ImportError as e:
    TIMESFM_AVAILABLE = False
    print("❌ TimesFM or PyTorch not available. Please install with:")
    print("   pip install timesfm torch")
    print("   # For CUDA support: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
    print("   # For covariates support, also install: pip install jax jaxlib")
    print(f"   Import error: {e}")

import joblib
import os
import json


 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0].
✅ TimesFM and PyTorch are available
   🔥 PyTorch version: 2.6.0+cu124
   🧠 CUDA available: True


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
class WalmartPreprocessingPipeline:
    """
    Complete preprocessing pipeline for Walmart sales data
    Supports fit/transform pattern for proper train/validation handling
    """

    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None
        self.feature_columns = None

    def load_and_prepare_data(self):
        """Load and merge train.csv, stores.csv, features.csv datasets"""
        print("📊 Loading datasets...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")
        print(f"   🎯 Features data: {features_df.shape}")

        # Convert Date column to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge datasets
        train_stores = train_df.merge(stores_df, on='Store', how='left')
        train_full = train_stores.merge(features_df, on=['Store', 'Date'], how='left')

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def clean_merged_data(self, train_full):
        """Clean merged data by handling duplicate IsHoliday columns"""
        print("🧹 Cleaning merged data...")

        initial_shape = train_full.shape

        # Handle duplicate IsHoliday columns if they exist
        if 'IsHoliday_x' in train_full.columns and 'IsHoliday_y' in train_full.columns:
            print("   🔄 Resolving duplicate IsHoliday columns...")
            train_full['IsHoliday'] = train_full['IsHoliday_x'] | train_full['IsHoliday_y']
            train_full = train_full.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

        print(f"   ✅ Cleaned data: {train_full.shape} (was {initial_shape})")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage"""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        # Sort by date to ensure temporal order
        df_sorted = df.sort_values('Date').reset_index(drop=True)

        # Find split point
        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']

        # Create splits
        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()

        # Create split info dictionary
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """Fit the preprocessing pipeline on training data"""
        print("🔧 Fitting preprocessing pipeline on training data...")

        # Store training data for lag feature creation
        self.train_data_for_lags = train_data.copy()

        # Fit outlier removal thresholds on training data only
        self.outlier_thresholds = {
            'A': {'lower': -1000, 'upper': 50000},  # Type A stores
            'B': {'lower': -500, 'upper': 25000},   # Type B stores
            'C': {'lower': -200, 'upper': 15000}    # Type C stores
        }

        print("✅ Pipeline fitted on training data")
        self.fitted = True
        return self

    def transform(self, data, is_validation=False):
        """Transform data using fitted pipeline"""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")

        print(f"🔄 Transforming {'validation' if is_validation else 'training'} data...")

        df = data.copy()

        # Step 1: Create date features
        df = self._create_date_features(df)

        # Step 2: Create holiday features
        df = self._create_holiday_features(df)

        # Step 3: Encode categorical features (BEFORE outlier removal!)
        df = self._encode_categorical_features(df)

        # Step 4: Remove outliers (only on training data, AFTER encoding)
        if not is_validation:
            df = self._remove_outliers(df)

        # Step 5: Remove markdown features
        df = self._remove_markdown_features(df)

        # Step 6: Remove redundant features
        df = self._remove_redundant_features(df)

        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        """Fit and transform training data in one step"""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _create_date_features(self, df):
        """Create date features"""
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['Quarter'] = df['Date'].dt.quarter
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        start_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - start_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        return df

    def _create_holiday_features(self, df):
        """Create holiday features"""
        df = df.copy()
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowl'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDay'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgiving'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmas'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)
        return df

    def _remove_outliers(self, df):
        """Remove outliers from training data"""
        initial_count = len(df)

        for store_type in ['A', 'B', 'C']:
            mask = df['Type'] == store_type
            if mask.sum() > 0:
                thresholds = self.outlier_thresholds[store_type]
                outlier_mask = (
                    (df['Weekly_Sales'] < thresholds['lower']) |
                    (df['Weekly_Sales'] > thresholds['upper'])
                )
                df = df[~(mask & outlier_mask)]

        removed_count = initial_count - len(df)
        print(f"   🧹 Removed {removed_count:,} outliers ({removed_count/initial_count*100:.1f}%)")
        return df

    def _remove_markdown_features(self, df):
        """Remove markdown features"""
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        if markdown_cols:
            df = df.drop(markdown_cols, axis=1)
            print(f"   📝 Removed {len(markdown_cols)} markdown features")
        return df

    def _remove_redundant_features(self, df):
        """Remove redundant features"""
        redundant_cols = ['Fuel_Price', 'CPI', 'Unemployment']
        existing_redundant = [col for col in redundant_cols if col in df.columns]
        if existing_redundant:
            df = df.drop(existing_redundant, axis=1)
            print(f"   🔄 Removed {len(existing_redundant)} redundant features")
        return df

    def _encode_categorical_features(self, df):
        """Encode categorical features"""
        df = df.copy()

        # Encode Type (A=0, B=1, C=2)
        if 'Type' in df.columns:
            type_mapping = {'A': 0, 'B': 1, 'C': 2}
            df['Type_Encoded'] = df['Type'].map(type_mapping)

        return df



In [20]:
def setup_mlflow():
    """Setup MLflow tracking with DagsHub"""
    print("🔧 Setting up MLflow tracking...")

    # DagsHub credentials (replace with your actual credentials)
    DAGSHUB_USERNAME = "konstantine25b"
    DAGSHUB_PROJECT = "Walmart-Recruiting---Store-Sales-Forecasting"

    # Configure DagsHub
    dagshub.init(repo_owner=DAGSHUB_USERNAME, repo_name=DAGSHUB_PROJECT, mlflow=True)

    experiment_name = "walmart_sales_forecasting_timesfm"

    try:
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            experiment_id = mlflow.create_experiment(experiment_name)
            print(f"   ✅ Created new experiment: {experiment_name}")
        else:
            experiment_id = experiment.experiment_id
            print(f"   ✅ Using existing experiment: {experiment_name}")
    except Exception as e:
        print(f"   ⚠️ Experiment setup issue: {e}")
        experiment_name = "Default"

    mlflow.set_experiment(experiment_name)
    print(f"   📊 MLflow experiment: {experiment_name}")

    return experiment_name





def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    """Calculate Weighted Mean Absolute Error (Competition Metric)"""
    weights = np.where(is_holiday, holiday_weight, 1.0)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)



In [21]:
def get_preprocessed_data():
    """Get preprocessed data using the pipeline"""
    print("📊 Getting preprocessed data...")

    # Initialize pipeline
    pipeline = WalmartPreprocessingPipeline()

    # Load and prepare data
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)

    # Create temporal split
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)

    # Fit pipeline on training data
    pipeline.fit(train_data)

    # Transform both training and validation data
    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data, is_validation=True)

    # Prepare features and targets
    feature_columns = [col for col in train_processed.columns
                      if col not in ['Date', 'Weekly_Sales', 'Store', 'Dept', 'Type']]

    X_train = train_processed[feature_columns]
    y_train = train_processed['Weekly_Sales']
    X_val = val_processed[feature_columns]
    y_val = val_processed['Weekly_Sales']

    # Get holiday information
    train_holidays = train_processed['IsHoliday']
    val_holidays = val_processed['IsHoliday']

    print(f"✅ Data preprocessing complete!")
    print(f"   📈 Training: {len(X_train):,} samples, {len(feature_columns)} features")
    print(f"   📉 Validation: {len(X_val):,} samples")

    return X_train, y_train, X_val, y_val, train_holidays, val_holidays, split_info, feature_columns, train_processed, val_processed



In [22]:
def initialize_timesfm_model():
    """Initialize TimesFM model with appropriate settings"""
    if not TIMESFM_AVAILABLE:
        raise ImportError("TimesFM is not available. Please install it first.")

    print("🤖 Initializing TimesFM model...")

    try:
        # Detect best available backend
        if torch.cuda.is_available():
            backend = "gpu"
            batch_size = 32
            print("   🚀 Using GPU backend")
        else:
            backend = "cpu"
            batch_size = 16
            print("   💻 Using CPU backend")

        # Initialize TimesFM with appropriate settings for weekly retail data
        tfm = timesfm.TimesFm(
            hparams=timesfm.TimesFmHparams(
                backend=backend,
                per_core_batch_size=batch_size,
                horizon_len=32,  # Forecast horizon (weeks)
                context_len=64,  # Context length (must be multiple of 32)
            ),
            checkpoint=timesfm.TimesFmCheckpoint(
                huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),  # Use PyTorch checkpoint
        )

        print("✅ TimesFM model initialized successfully")
        print(f"   📊 Backend: {backend}")
        print(f"   📦 Batch size: {batch_size}")
        print(f"   📈 Context length: 64 weeks")
        print(f"   🔮 Horizon length: 32 weeks")
        return tfm

    except Exception as e:
        print(f"❌ Failed to initialize TimesFM: {e}")
        print("   Make sure you have installed timesfm and its dependencies:")
        print("   pip install timesfm torch")
        print("   # For CUDA: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
        raise



In [23]:
def prepare_timesfm_data(train_data, val_data, train_holidays, val_holidays):
    """Prepare data for TimesFM forecasting"""
    print("📊 Preparing TimesFM data...")

    # Combine train and validation for full time series
    full_data = pd.concat([train_data, val_data]).reset_index(drop=True)
    full_holidays = pd.concat([train_holidays, val_holidays]).reset_index(drop=True)

    # Get unique Store-Dept combinations from regular DataFrame columns
    train_combinations = set(zip(train_data['Store'], train_data['Dept']))
    val_combinations = set(zip(val_data['Store'], val_data['Dept']))

    print(f"   📈 Train combinations: {len(train_combinations):,}")
    print(f"   📉 Val combinations: {len(val_combinations):,}")
    print(f"   🔄 Common combinations: {len(train_combinations & val_combinations):,}")

    return full_data, full_holidays, train_combinations, val_combinations



In [24]:
def train_timesfm_models(full_data, train_combinations, val_combinations, max_models=None):
    """Train TimesFM models for each Store-Dept combination"""
    print("🚀 Training TimesFM models...")

    # Initialize TimesFM
    tfm = initialize_timesfm_model()

    # Get combinations to process
    combinations_to_process = list(train_combinations & val_combinations)

    if max_models:
        combinations_to_process = combinations_to_process[:max_models]
        print(f"   🔧 Limiting to {max_models} models for testing")

    print(f"   🎯 Processing {len(combinations_to_process):,} Store-Dept combinations")

    # Prepare time series data
    time_series_data = []
    combination_info = []
    successful_models = 0
    skipped_models = 0
    failed_models = 0

    start_time = time.time()

    for i, (store, dept) in enumerate(combinations_to_process):
        try:
            # Filter data for this Store-Dept combination
            store_dept_data = full_data[
                (full_data['Store'] == store) &
                (full_data['Dept'] == dept)
            ].sort_values('Date')

            if len(store_dept_data) < 10:  # Minimum data requirement
                skipped_models += 1
                continue

            # Extract time series
            sales_series = store_dept_data['Weekly_Sales'].values

            # Store the series and combination info
            time_series_data.append(sales_series)
            combination_info.append((store, dept))
            successful_models += 1

            # Progress reporting
            if (i + 1) % 50 == 0:
                elapsed = time.time() - start_time
                speed = (i + 1) / elapsed
                remaining = len(combinations_to_process) - (i + 1)
                eta_seconds = remaining / speed if speed > 0 else 0
                eta_minutes = eta_seconds / 60

                print(f"   📊 Progress: {i+1:,}/{len(combinations_to_process):,} "
                      f"({(i+1)/len(combinations_to_process)*100:.1f}%) | "
                      f"✅ Success: {successful_models:,} | "
                      f"⏭️ Skipped: {skipped_models:,} | "
                      f"❌ Failed: {failed_models:,} | "
                      f"⚡ Speed: {speed:.1f} comb/sec | "
                      f"⏱️ ETA: {eta_minutes:.1f}min")

        except Exception as e:
            failed_models += 1
            continue

    print(f"✅ TimesFM data preparation complete!")
    print(f"   🎯 Successful models: {successful_models:,}")
    print(f"   ⏭️ Skipped models: {skipped_models:,}")
    print(f"   ❌ Failed models: {failed_models:,}")

    return tfm, time_series_data, combination_info



In [25]:
def make_timesfm_predictions(tfm, time_series_data, combination_info, full_data, val_data):
    """Make predictions using trained TimesFM model"""
    print("🔮 Making TimesFM predictions...")

    if len(time_series_data) == 0:
        print("❌ No time series data available for predictions")
        return np.array([]), np.array([]), np.array([])

    predictions = []
    actuals = []
    holidays = []

    # Determine forecast horizon (number of validation weeks)
    forecast_horizon = len(val_data['Date'].unique())

    try:
        # Set frequency for weekly retail data (using medium frequency)
        frequency_input = [1] * len(time_series_data)  # Medium frequency for weekly data

        print(f"   🔮 Forecasting {forecast_horizon} weeks for {len(time_series_data):,} time series...")

        # Make forecasts using TimesFM
        point_forecast, experimental_quantile_forecast = tfm.forecast(
            time_series_data,
            freq=frequency_input,
        )

        print(f"   ✅ TimesFM forecasting complete!")
        print(f"   📊 Forecast shape: {point_forecast.shape}")

        # Extract predictions for validation period
        for i, (store, dept) in enumerate(combination_info):
            try:
                # Get validation data for this Store-Dept
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ].sort_values('Date')

                if len(store_dept_val) == 0:
                    continue

                # Get forecasted values
                forecast_values = point_forecast[i][:len(store_dept_val)]

                # Store results
                predictions.extend(forecast_values.tolist())
                actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                holidays.extend(store_dept_val['IsHoliday'].tolist())

            except Exception as e:
                # Fallback prediction
                fallback_pred = val_data['Weekly_Sales'].mean()
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ]
                if len(store_dept_val) > 0:
                    predictions.extend([fallback_pred] * len(store_dept_val))
                    actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                    holidays.extend(store_dept_val['IsHoliday'].tolist())

    except Exception as e:
        print(f"❌ TimesFM prediction failed: {e}")
        print("   🔄 Using fallback predictions...")

        # Fallback to mean prediction
        fallback_pred = val_data['Weekly_Sales'].mean()
        predictions = [fallback_pred] * len(val_data)
        actuals = val_data['Weekly_Sales'].tolist()
        holidays = val_data['IsHoliday'].tolist()

    print(f"✅ Predictions complete!")
    print(f"   🎯 Total predictions: {len(predictions):,}")

    return np.array(predictions), np.array(actuals), np.array(holidays)



In [26]:
def main():
    """Main experiment execution"""
    print("🚀 Starting Experiment 12: TimesFM (Time Series Foundation Model)")
    print("=" * 80)

    # Check TimesFM availability
    if not TIMESFM_AVAILABLE:
        print("❌ TimesFM is not available. Please install it first:")
        print("   pip install timesfm torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q")
        print("   # For CUDA support: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
        print("   # For covariates support, also install: pip install jax jaxlib")
        return

    # Setup MLflow tracking
    experiment_name = setup_mlflow()

    with mlflow.start_run(run_name="TimesFM_Foundation_Model") as run:
        print(f"🔄 Starting MLflow run: {run.info.run_id}")

        # Log experiment metadata
        mlflow.log_param("experiment_type", "TimesFM_Foundation_Model")
        mlflow.log_param("model_type", "TimesFM_Pretrained")
        mlflow.log_param("feature_engineering", "Experiment_7_Pipeline")
        mlflow.log_param("data_split", "temporal_80_20")
        mlflow.log_param("foundation_model", "google/timesfm-1.0-200m-pytorch")
        mlflow.log_param("approach", "Pretrained_Foundation_Model")

        try:
            # Step 1: Get preprocessed data
            print("\n📊 Step 1: Data preprocessing...")
            X_train, y_train, X_val, y_val, train_holidays, val_holidays, split_info, feature_columns, train_processed, val_processed = get_preprocessed_data()

            # Reconstruct train and val dataframes for TimesFM
            # We need to get the original processed data with Store and Dept columns
            train_data = train_processed[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].copy()
            val_data = val_processed[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].copy()

            # Log data info
            mlflow.log_metric("train_samples", len(train_data))
            mlflow.log_metric("val_samples", len(val_data))
            mlflow.log_metric("total_features", len(feature_columns))
            mlflow.log_param("split_date", str(split_info['split_date']))

            # Step 2: Prepare TimesFM data
            print("\n📊 Step 2: Preparing TimesFM data...")
            full_data, full_holidays, train_combinations, val_combinations = prepare_timesfm_data(
                train_data, val_data, train_holidays, val_holidays
            )

            # Log combination info
            mlflow.log_metric("train_combinations", len(train_combinations))
            mlflow.log_metric("val_combinations", len(val_combinations))
            mlflow.log_metric("common_combinations", len(train_combinations & val_combinations))

            # Step 3: Train TimesFM models
            print("\n🤖 Step 3: Training TimesFM models...")

            # OPTION: Limit models for faster testing (uncomment next line for testing)
            max_models_for_testing = 100  # Test with only 100 models for speed
            # max_models_for_testing = None   # Use None for full training

            tfm, time_series_data, combination_info = train_timesfm_models(
                full_data, train_combinations, val_combinations, max_models=max_models_for_testing
            )

            # Log training info
            mlflow.log_metric("successful_models", len(time_series_data))
            mlflow.log_metric("processed_combinations", len(combination_info))

            # Step 4: Make predictions
            print("\n🔮 Step 4: Making predictions...")
            y_pred, y_true, is_holiday = make_timesfm_predictions(
                tfm, time_series_data, combination_info, full_data, val_data
            )

            # Step 5: Calculate metrics
            print("\n📊 Step 5: Calculating metrics...")

            if len(y_pred) > 0 and len(y_true) > 0:
                # Standard metrics
                mae = mean_absolute_error(y_true, y_pred)
                rmse = np.sqrt(mean_squared_error(y_true, y_pred))
                r2 = r2_score(y_true, y_pred)

                # WMAE (Competition metric)
                wmae = calculate_wmae(y_true, y_pred, is_holiday)

                # Holiday vs non-holiday breakdown
                holiday_mask = is_holiday == True
                non_holiday_mask = is_holiday == False

                holiday_mae = mean_absolute_error(y_true[holiday_mask], y_pred[holiday_mask]) if holiday_mask.sum() > 0 else 0
                non_holiday_mae = mean_absolute_error(y_true[non_holiday_mask], y_pred[non_holiday_mask]) if non_holiday_mask.sum() > 0 else 0

                # Log all metrics
                mlflow.log_metric("mae", mae)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2_score", r2)
                mlflow.log_metric("wmae", wmae)
                mlflow.log_metric("holiday_mae", holiday_mae)
                mlflow.log_metric("non_holiday_mae", non_holiday_mae)
                mlflow.log_metric("holiday_samples", holiday_mask.sum())
                mlflow.log_metric("non_holiday_samples", non_holiday_mask.sum())

                # Step 6: Results summary
                print("\n" + "=" * 60)
                print("🎯 EXPERIMENT 12 RESULTS SUMMARY")
                print("=" * 60)
                print(f"📊 Validation Metrics:")
                print(f"   WMAE (Competition Metric): ${wmae:,.2f}")
                print(f"   MAE: ${mae:,.2f}")
                print(f"   RMSE: ${rmse:,.2f}")
                print(f"   R²: {r2:.4f}")
                print(f"\n📊 Holiday Breakdown:")
                print(f"   Holiday MAE: ${holiday_mae:,.2f} ({holiday_mask.sum():,} samples)")
                print(f"   Non-Holiday MAE: ${non_holiday_mae:,.2f} ({non_holiday_mask.sum():,} samples)")
                print(f"\n📊 Model Statistics:")
                print(f"   Time series processed: {len(time_series_data):,}")
                print(f"   Store-Dept combinations: {len(train_combinations):,}")
                print(f"   Foundation model: google/timesfm-1.0-200m-pytorch")

                # Step 7: Save artifacts
                print(f"\n💾 Saving model artifacts...")

                # Save model summary
                model_summary = {
                    'experiment_name': experiment_name,
                    'run_id': run.info.run_id,
                    'time_series_processed': len(time_series_data),
                    'foundation_model': 'google/timesfm-1.0-200m-pytorch',
                    'feature_columns': feature_columns,
                    'metrics': {
                        'wmae': wmae,
                        'mae': mae,
                        'rmse': rmse,
                        'r2': r2
                    }
                }

                with open('timesfm_exp12_summary.json', 'w') as f:
                    json.dump(model_summary, f, indent=2, default=str)

                mlflow.log_artifact('timesfm_exp12_summary.json')

                print("✅ Experiment 12 completed successfully!")

            else:
                print("❌ No predictions generated - check data and model setup")
                mlflow.log_param("error", "No predictions generated")

        except Exception as e:
            print(f"❌ Experiment failed: {e}")
            mlflow.log_param("error", str(e))
            raise

    print("\n🎉 Experiment 12: TimesFM Foundation Model - COMPLETE!")



In [27]:
if __name__ == "__main__":
    main()

🚀 Starting Experiment 12: TimesFM (Time Series Foundation Model)
🔧 Setting up MLflow tracking...


Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

   ✅ Using existing experiment: walmart_sales_forecasting_timesfm
   📊 MLflow experiment: walmart_sales_forecasting_timesfm
🔄 Starting MLflow run: cbf4d30950aa4d08a80e00ba6edef3a3

📊 Step 1: Data preprocessing...
📊 Getting preprocessed data...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data
🔄 Transforming training data...
   🧹 Removed 45,193 outliers (13.4%)
   📝 Removed 5 markdown features
   🔄 Removed 3 redundant fe

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

torch_model.ckpt:   0%|          | 0.00/814M [00:00<?, ?B/s]

✅ TimesFM model initialized successfully
   📊 Backend: gpu
   📦 Batch size: 32
   📈 Context length: 64 weeks
   🔮 Horizon length: 32 weeks
   🔧 Limiting to 100 models for testing
   🎯 Processing 100 Store-Dept combinations
   📊 Progress: 50/100 (50.0%) | ✅ Success: 50 | ⏭️ Skipped: 0 | ❌ Failed: 0 | ⚡ Speed: 770.0 comb/sec | ⏱️ ETA: 0.0min
   📊 Progress: 100/100 (100.0%) | ✅ Success: 100 | ⏭️ Skipped: 0 | ❌ Failed: 0 | ⚡ Speed: 799.4 comb/sec | ⏱️ ETA: 0.0min
✅ TimesFM data preparation complete!
   🎯 Successful models: 100
   ⏭️ Skipped models: 0
   ❌ Failed models: 0

🔮 Step 4: Making predictions...
🔮 Making TimesFM predictions...
   🔮 Forecasting 29 weeks for 100 time series...
   ✅ TimesFM forecasting complete!
   📊 Forecast shape: (100, 32)
✅ Predictions complete!
   🎯 Total predictions: 2,722

📊 Step 5: Calculating metrics...

🎯 EXPERIMENT 12 RESULTS SUMMARY
📊 Validation Metrics:
   WMAE (Competition Metric): $1,985.27
   MAE: $1,976.18
   RMSE: $4,068.65
   R²: 0.9131

📊 Holiday 

In [28]:
def initialize_timesfm_model():
    """Initialize TimesFM model with appropriate settings"""
    if not TIMESFM_AVAILABLE:
        raise ImportError("TimesFM is not available. Please install it first.")

    print("🤖 Initializing TimesFM model...")

    try:
        # Detect best available backend
        if torch.cuda.is_available():
            backend = "gpu"
            batch_size = 32
            print("   🚀 Using GPU backend")
        else:
            backend = "cpu"
            batch_size = 16
            print("   💻 Using CPU backend")

        # Initialize TimesFM with enhanced settings for retail data
        tfm = timesfm.TimesFm(
            hparams=timesfm.TimesFmHparams(
                backend=backend,
                per_core_batch_size=batch_size,
                horizon_len=64,  # Increased horizon for longer forecasts
                context_len=128,  # Increased context for better pattern recognition
            ),
            checkpoint=timesfm.TimesFmCheckpoint(
                huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),  # Use PyTorch checkpoint
        )

        print("✅ TimesFM model initialized successfully")
        print(f"   📊 Backend: {backend}")
        print(f"   📦 Batch size: {batch_size}")
        print(f"   📈 Context length: 128 weeks")
        print(f"   🔮 Horizon length: 64 weeks")
        return tfm

    except Exception as e:
        print(f"❌ Failed to initialize TimesFM: {e}")
        print("   Make sure you have installed timesfm and its dependencies:")
        print("   pip install timesfm torch")
        print("   # For CUDA: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
        raise


In [29]:
def initialize_multiple_timesfm_models():
    """Initialize multiple TimesFM models with different configurations for ensemble"""
    if not TIMESFM_AVAILABLE:
        raise ImportError("TimesFM is not available. Please install it first.")

    print("🤖 Initializing multiple TimesFM model configurations...")

    try:
        # Detect best available backend
        if torch.cuda.is_available():
            backend = "gpu"
            base_batch_size = 32
            print("   🚀 Using GPU backend")
        else:
            backend = "cpu"
            base_batch_size = 16
            print("   💻 Using CPU backend")

        models = {}

        # Configuration 1: Standard model (balanced)
        models['standard'] = timesfm.TimesFm(
            hparams=timesfm.TimesFmHparams(
                backend=backend,
                per_core_batch_size=base_batch_size,
                horizon_len=64,
                context_len=128,
            ),
            checkpoint=timesfm.TimesFmCheckpoint(
                huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
        )

        # Configuration 2: Long context model (for complex patterns)
        models['long_context'] = timesfm.TimesFm(
            hparams=timesfm.TimesFmHparams(
                backend=backend,
                per_core_batch_size=max(base_batch_size // 2, 8),  # Reduce batch size for longer context
                horizon_len=32,
                context_len=256,  # Longer context for complex patterns
            ),
            checkpoint=timesfm.TimesFmCheckpoint(
                huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
        )

        # Configuration 3: Short horizon model (for immediate predictions)
        models['short_horizon'] = timesfm.TimesFm(
            hparams=timesfm.TimesFmHparams(
                backend=backend,
                per_core_batch_size=base_batch_size * 2,  # Increase batch size for shorter horizon
                horizon_len=16,
                context_len=64,
            ),
            checkpoint=timesfm.TimesFmCheckpoint(
                huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
        )

        print(f"✅ {len(models)} TimesFM model configurations initialized:")
        for name, _ in models.items():
            print(f"   📊 {name}: ready")

        return models

    except Exception as e:
        print(f"❌ Failed to initialize TimesFM models: {e}")
        print("   Falling back to single model configuration...")
        return {'standard': initialize_timesfm_model()}


In [30]:
def prepare_advanced_timesfm_data(full_data, train_combinations, val_combinations, max_models=None):
    """Prepare advanced time series data with multiple configurations and covariates"""
    print("🚀 Preparing advanced TimesFM data with covariates...")

    # Get combinations to process
    combinations_to_process = list(train_combinations & val_combinations)

    if max_models:
        # Sort by data volume for better representative sample
        combo_stats = []
        for store, dept in combinations_to_process:
            store_dept_data = full_data[
                (full_data['Store'] == store) &
                (full_data['Dept'] == dept)
            ]
            combo_stats.append((store, dept, len(store_dept_data)))

        # Sort by data volume (descending) and take top combinations
        combo_stats.sort(key=lambda x: x[2], reverse=True)
        combinations_to_process = [(store, dept) for store, dept, _ in combo_stats[:max_models]]
        print(f"   🔧 Limiting to {max_models} models (selected by data volume)")

    print(f"   🎯 Processing {len(combinations_to_process):,} Store-Dept combinations")

    # Prepare multiple time series configurations
    time_series_configs = {
        'raw_sales': [],
        'log_sales': [],
        'normalized_sales': [],
        'detrended_sales': []
    }

    # Prepare covariates data
    static_categorical_covariates = []
    static_numerical_covariates = []
    dynamic_categorical_covariates = []
    dynamic_numerical_covariates = []

    combination_info = []
    successful_models = 0
    skipped_models = 0
    failed_models = 0

    start_time = time.time()

    for i, (store, dept) in enumerate(combinations_to_process):
        try:
            # Filter data for this Store-Dept combination
            store_dept_data = full_data[
                (full_data['Store'] == store) &
                (full_data['Dept'] == dept)
            ].sort_values('Date')

            if len(store_dept_data) < 20:  # Increased minimum data requirement
                skipped_models += 1
                continue

            # Extract raw sales
            sales_series = store_dept_data['Weekly_Sales'].values

            # Configuration 1: Raw sales
            time_series_configs['raw_sales'].append(sales_series)

            # Configuration 2: Log-transformed sales (handle zeros)
            log_sales = np.log1p(np.maximum(sales_series, 0))
            time_series_configs['log_sales'].append(log_sales)

            # Configuration 3: Normalized sales (z-score)
            if sales_series.std() > 0:
                normalized_sales = (sales_series - sales_series.mean()) / sales_series.std()
            else:
                normalized_sales = sales_series
            time_series_configs['normalized_sales'].append(normalized_sales)

            # Configuration 4: Detrended sales
            x = np.arange(len(sales_series))
            if len(x) > 1:
                trend_coef = np.polyfit(x, sales_series, 1)[0]
                detrended_sales = sales_series - trend_coef * x
            else:
                detrended_sales = sales_series
            time_series_configs['detrended_sales'].append(detrended_sales)

            # Prepare static covariates
            store_type = store_dept_data['Type_Encoded'].iloc[0] if 'Type_Encoded' in store_dept_data.columns else 0
            store_size = store_dept_data['Size'].iloc[0] if 'Size' in store_dept_data.columns else 0

            static_categorical_covariates.append([store_type])  # Store type
            static_numerical_covariates.append([store_size, store, dept])  # Store size, Store ID, Dept ID

            # Prepare dynamic covariates (aligned with time series length)
            dynamic_cat_data = []
            dynamic_num_data = []

            for _, row in store_dept_data.iterrows():
                # Dynamic categorical: IsHoliday, Month, Quarter, DayOfWeek
                dynamic_cat_data.append([
                    int(row.get('IsHoliday', 0)),
                    int(row.get('Month', 1)),
                    int(row.get('Quarter', 1)),
                    int(row.get('DayOfWeek', 0))
                ])

                # Dynamic numerical: Temperature, Size, WeekOfYear, IsWeekend
                dynamic_num_data.append([
                    float(row.get('Temperature', 70.0)),
                    float(row.get('Size', 0.0)),
                    float(row.get('WeekOfYear', 1.0)),
                    float(row.get('IsWeekend', 0.0))
                ])

            dynamic_categorical_covariates.append(dynamic_cat_data)
            dynamic_numerical_covariates.append(dynamic_num_data)

            combination_info.append((store, dept))
            successful_models += 1

            # Progress reporting
            if (i + 1) % 50 == 0:
                elapsed = time.time() - start_time
                speed = (i + 1) / elapsed
                remaining = len(combinations_to_process) - (i + 1)
                eta_seconds = remaining / speed if speed > 0 else 0
                eta_minutes = eta_seconds / 60

                print(f"   📊 Progress: {i+1:,}/{len(combinations_to_process):,} "
                      f"({(i+1)/len(combinations_to_process)*100:.1f}%) | "
                      f"✅ Success: {successful_models:,} | "
                      f"⏭️ Skipped: {skipped_models:,} | "
                      f"❌ Failed: {failed_models:,} | "
                      f"⚡ Speed: {speed:.1f} comb/sec | "
                      f"⏱️ ETA: {eta_minutes:.1f}min")

        except Exception as e:
            failed_models += 1
            print(f"   ⚠️ Failed to process Store {store}, Dept {dept}: {e}")
            continue

    print(f"✅ Advanced TimesFM data preparation complete!")
    print(f"   🎯 Successful models: {successful_models:,}")
    print(f"   ⏭️ Skipped models: {skipped_models:,}")
    print(f"   ❌ Failed models: {failed_models:,}")
    print(f"   📊 Time series configurations: {len(time_series_configs)}")
    print(f"   🎛️ Static covariates: categorical={len(static_categorical_covariates[0]) if static_categorical_covariates else 0}, numerical={len(static_numerical_covariates[0]) if static_numerical_covariates else 0}")
    print(f"   🎛️ Dynamic covariates: categorical={len(dynamic_categorical_covariates[0][0]) if dynamic_categorical_covariates else 0}, numerical={len(dynamic_numerical_covariates[0][0]) if dynamic_numerical_covariates else 0}")

    covariates_data = {
        'static_categorical': static_categorical_covariates,
        'static_numerical': static_numerical_covariates,
        'dynamic_categorical': dynamic_categorical_covariates,
        'dynamic_numerical': dynamic_numerical_covariates
    }

    return time_series_configs, combination_info, covariates_data



In [36]:
def make_ensemble_timesfm_predictions(tfm, time_series_configs, combination_info, covariates_data, full_data, val_data):
    """Make ensemble predictions using multiple TimesFM configurations"""
    print("🔮 Making ensemble TimesFM predictions...")

    if len(time_series_configs['raw_sales']) == 0:
        print("❌ No time series data available for predictions")
        return np.array([]), np.array([]), np.array([])

    predictions = []
    actuals = []
    holidays = []

    # Determine forecast horizon
    forecast_horizon = len(val_data['Date'].unique())

    # Store predictions from different configurations
    all_predictions = {}

    try:
        print(f"   🔮 Running ensemble forecasting with {len(time_series_configs)} configurations...")

        # Run predictions for each configuration
        for config_name, time_series_data in time_series_configs.items():
            print(f"   📊 Forecasting with {config_name} configuration...")

            try:
                # Set frequency for weekly retail data
                frequency_input = [1] * len(time_series_data)  # Medium frequency for weekly data

                # Make forecasts using TimesFM
                if config_name == 'raw_sales':
                    # For raw sales, try to use covariates if available
                    try:
                        # Attempt to use covariates (requires JAX installation)
                        point_forecast, experimental_quantile_forecast = tfm.forecast_with_covariates(
                            time_series_data,
                            freq=frequency_input,
                            static_categorical_covariates=covariates_data['static_categorical'],
                            static_numerical_covariates=covariates_data['static_numerical'],
                            dynamic_categorical_covariates=covariates_data['dynamic_categorical'],
                            dynamic_numerical_covariates=covariates_data['dynamic_numerical'],
                        )
                        print(f"     ✅ Using covariates for {config_name}")
                    except:
                        # Fallback to standard forecast if covariates fail
                        point_forecast, experimental_quantile_forecast = tfm.forecast(
                            time_series_data,
                            freq=frequency_input,
                        )
                        print(f"     ⚠️ Covariates failed, using standard forecast for {config_name}")
                else:
                    # Standard forecast for other configurations
                    point_forecast, experimental_quantile_forecast = tfm.forecast(
                        time_series_data,
                        freq=frequency_input,
                    )

                all_predictions[config_name] = point_forecast
                print(f"     ✅ {config_name} forecasting complete! Shape: {point_forecast.shape}")

            except Exception as e:
                print(f"     ❌ {config_name} forecasting failed: {e}")
                continue

        if not all_predictions:
            raise Exception("All forecasting configurations failed")

        # Ensemble predictions by averaging
        print(f"   🎯 Creating ensemble from {len(all_predictions)} successful configurations...")

        # Extract predictions for validation period
        for i, (store, dept) in enumerate(combination_info):
            try:
                # Get validation data for this Store-Dept
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ].sort_values('Date')

                if len(store_dept_val) == 0:
                    continue

                # Collect predictions from all configurations
                config_predictions = []
                for config_name, forecast in all_predictions.items():
                    if i < len(forecast):
                        pred_values = forecast[i][:len(store_dept_val)]

                        # Post-process based on configuration
                        if config_name == 'log_sales':
                            # Inverse log transform
                            pred_values = np.expm1(pred_values)
                        elif config_name == 'normalized_sales':
                            # Denormalize (need original stats)
                            original_data = full_data[
                                (full_data['Store'] == store) &
                                (full_data['Dept'] == dept)
                            ]['Weekly_Sales'].values
                            if len(original_data) > 0 and original_data.std() > 0:
                                pred_values = pred_values * original_data.std() + original_data.mean()
                        elif config_name == 'detrended_sales':
                            # Re-add trend
                            original_data = full_data[
                                (full_data['Store'] == store) &
                                (full_data['Dept'] == dept)
                            ]['Weekly_Sales'].values
                            if len(original_data) > 1:
                                x_trend = np.arange(len(original_data), len(original_data) + len(pred_values))
                                trend_coef = np.polyfit(np.arange(len(original_data)), original_data, 1)[0]
                                pred_values = pred_values + trend_coef * x_trend

                        config_predictions.append(pred_values)

                if config_predictions:
                    # Ensemble by averaging (with optional weighting)
                    weights = [1.0, 0.8, 0.6, 0.7]  # Custom weights for each configuration
                    weights = weights[:len(config_predictions)]
                    weights = np.array(weights) / np.sum(weights)  # Normalize weights

                    ensemble_pred = np.average(config_predictions, axis=0, weights=weights)

                    # Apply post-processing smoothing (optional)
                    if len(ensemble_pred) > 2:
                        # Simple moving average smoothing
                        smoothed_pred = np.convolve(ensemble_pred, [0.2, 0.6, 0.2], mode='same')
                        ensemble_pred = smoothed_pred

                    # Ensure we match the validation length
                    final_pred_length = min(len(ensemble_pred), len(store_dept_val))
                    final_pred = ensemble_pred[:final_pred_length]
                    final_actual = store_dept_val['Weekly_Sales'].iloc[:final_pred_length].tolist()
                    final_holiday = store_dept_val['IsHoliday'].iloc[:final_pred_length].tolist()

                    predictions.extend(final_pred.tolist())
                    actuals.extend(final_actual)
                    holidays.extend(final_holiday)
                else:
                    # Fallback prediction
                    fallback_pred = val_data['Weekly_Sales'].mean()
                    predictions.extend([fallback_pred] * len(store_dept_val))
                    actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                    holidays.extend(store_dept_val['IsHoliday'].tolist())

            except Exception as e:
                print(f"     ⚠️ Error processing Store {store}, Dept {dept}: {e}")
                # Fallback prediction
                fallback_pred = val_data['Weekly_Sales'].mean()
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ]
                if len(store_dept_val) > 0:
                    predictions.extend([fallback_pred] * len(store_dept_val))
                    actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                    holidays.extend(store_dept_val['IsHoliday'].tolist())

    except Exception as e:
        print(f"❌ TimesFM ensemble prediction failed: {e}")
        print("   🔄 Using fallback predictions...")

        # Fallback to mean prediction
        fallback_pred = val_data['Weekly_Sales'].mean()
        predictions = [fallback_pred] * len(val_data)
        actuals = val_data['Weekly_Sales'].tolist()
        holidays = val_data['IsHoliday'].tolist()

    print(f"✅ Ensemble predictions complete!")
    print(f"   🎯 Total predictions: {len(predictions):,}")
    print(f"   📊 Configurations used: {len(all_predictions)}")

    return np.array(predictions), np.array(actuals), np.array(holidays)


In [37]:
def make_advanced_ensemble_predictions(models, time_series_configs, combination_info, covariates_data, full_data, val_data):
    """Make advanced ensemble predictions using multiple models and configurations"""
    print("🔮 Making advanced ensemble predictions...")

    if len(time_series_configs['raw_sales']) == 0:
        print("❌ No time series data available for predictions")
        return np.array([]), np.array([]), np.array([])

    predictions = []
    actuals = []
    holidays = []

    # Determine forecast horizon
    forecast_horizon = len(val_data['Date'].unique())

    # Store predictions from different model-configuration combinations
    all_predictions = {}

    try:
        print(f"   🔮 Running advanced ensemble with {len(models)} models and {len(time_series_configs)} configurations...")

        # Run predictions for each model-configuration combination
        for model_name, tfm in models.items():
            print(f"   🤖 Using {model_name} model...")

            for config_name, time_series_data in time_series_configs.items():
                combo_key = f"{model_name}_{config_name}"
                print(f"     📊 Forecasting with {combo_key}...")

                try:
                    # Set frequency for weekly retail data
                    frequency_input = [1] * len(time_series_data)

                    # Make forecasts based on configuration
                    if config_name == 'raw_sales' and model_name == 'standard':
                        # Try covariates only for standard model with raw sales
                        try:
                            point_forecast, _ = tfm.forecast_with_covariates(
                                time_series_data,
                                freq=frequency_input,
                                static_categorical_covariates=covariates_data['static_categorical'],
                                static_numerical_covariates=covariates_data['static_numerical'],
                                dynamic_categorical_covariates=covariates_data['dynamic_categorical'],
                                dynamic_numerical_covariates=covariates_data['dynamic_numerical'],
                            )
                            print(f"       ✅ Using covariates for {combo_key}")
                        except:
                            point_forecast, _ = tfm.forecast(time_series_data, freq=frequency_input)
                            print(f"       ⚠️ Covariates failed, using standard forecast for {combo_key}")
                    else:
                        # Standard forecast for other combinations
                        point_forecast, _ = tfm.forecast(time_series_data, freq=frequency_input)

                    all_predictions[combo_key] = point_forecast
                    print(f"       ✅ {combo_key} complete! Shape: {point_forecast.shape}")

                except Exception as e:
                    print(f"       ❌ {combo_key} failed: {e}")
                    continue

        if not all_predictions:
            raise Exception("All model-configuration combinations failed")

        # Advanced ensemble with performance weighting
        print(f"   🎯 Creating advanced ensemble from {len(all_predictions)} successful combinations...")

        # Debug: Print forecast shapes for troubleshooting
        print(f"   🔍 Debug: Forecast shapes:")
        for combo_key, forecast in all_predictions.items():
            print(f"     {combo_key}: {forecast.shape}")

        # Model performance weights (based on expected performance)
        model_weights = {
            'standard': 1.0,
            'long_context': 0.9,
            'short_horizon': 0.8
        }

        # Configuration weights (based on transformation stability)
        config_weights = {
            'raw_sales': 1.0,
            'log_sales': 0.8,
            'normalized_sales': 0.6,
            'detrended_sales': 0.7
        }

        # Get the target forecast length from validation data
        target_forecast_length = len(val_data['Date'].unique())
        print(f"   🎯 Target forecast length: {target_forecast_length} weeks")

        # Extract predictions for validation period
        for i, (store, dept) in enumerate(combination_info):
            try:
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ].sort_values('Date')

                if len(store_dept_val) == 0:
                    continue

                # Collect weighted predictions from all combinations
                weighted_predictions = []
                weights = []

                for combo_key, forecast in all_predictions.items():
                    if i < len(forecast):
                        model_name, config_name = combo_key.split('_', 1)

                        # Get the forecast for this time series
                        forecast_values = forecast[i]

                        # Ensure we only take the length we need for validation
                        # Truncate to the actual validation period length for this Store-Dept
                        pred_length = min(len(store_dept_val), len(forecast_values))
                        pred_values = forecast_values[:pred_length]

                        # Convert to numpy array to handle potential shape issues
                        pred_values = np.array(pred_values).flatten()

                        # Ensure exact length match - pad with last value if needed, truncate if too long
                        target_length = len(store_dept_val)
                        if len(pred_values) < target_length:
                            # Pad with last value if forecast is shorter
                            last_value = pred_values[-1] if len(pred_values) > 0 else 0
                            padding = np.full(target_length - len(pred_values), last_value)
                            pred_values = np.concatenate([pred_values, padding])
                        elif len(pred_values) > target_length:
                            # Truncate if forecast is longer
                            pred_values = pred_values[:target_length]

                        # Final validation - skip if still doesn't match
                        if len(pred_values) != target_length:
                            print(f"       ⚠️ Skipping {combo_key}: length mismatch {len(pred_values)} vs {target_length}")
                            continue

                        # Only proceed if we have the right length
                        if len(pred_values) != target_length:
                            continue

                        # Post-process based on configuration
                        if config_name == 'log_sales':
                            pred_values = np.expm1(pred_values)
                        elif config_name == 'normalized_sales':
                            original_data = full_data[
                                (full_data['Store'] == store) &
                                (full_data['Dept'] == dept)
                            ]['Weekly_Sales'].values
                            if len(original_data) > 0 and original_data.std() > 0:
                                pred_values = pred_values * original_data.std() + original_data.mean()
                        elif config_name == 'detrended_sales':
                            original_data = full_data[
                                (full_data['Store'] == store) &
                                (full_data['Dept'] == dept)
                            ]['Weekly_Sales'].values
                            if len(original_data) > 1:
                                x_trend = np.arange(len(original_data), len(original_data) + len(pred_values))
                                trend_coef = np.polyfit(np.arange(len(original_data)), original_data, 1)[0]
                                pred_values = pred_values + trend_coef * x_trend

                        # Calculate combination weight
                        combo_weight = model_weights.get(model_name, 0.5) * config_weights.get(config_name, 0.5)

                        weighted_predictions.append(pred_values)
                        weights.append(combo_weight)

                if weighted_predictions and len(weighted_predictions) > 0:
                    try:
                        # Convert all predictions to numpy arrays and ensure consistent shape
                        aligned_predictions = []
                        for pred in weighted_predictions:
                            pred_array = np.array(pred).flatten()
                            aligned_predictions.append(pred_array)

                        # Double-check all predictions have the same length
                        lengths = [len(pred) for pred in aligned_predictions]
                        if len(set(lengths)) > 1:
                            print(f"       ⚠️ Length mismatch in predictions: {lengths}")
                            # Force align to minimum length
                            min_length = min(lengths)
                            aligned_predictions = [pred[:min_length] for pred in aligned_predictions]
                            # Update store_dept_val to match
                            store_dept_val = store_dept_val.iloc[:min_length]

                        # Weighted ensemble
                        weights = np.array(weights) / np.sum(weights)  # Normalize weights

                        # Use numpy average with proper axis handling
                        ensemble_pred = np.average(np.array(aligned_predictions), axis=0, weights=weights)

                        # Apply post-processing smoothing (optional)
                        if len(ensemble_pred) > 2:
                            # Simple moving average smoothing
                            try:
                                smoothed_pred = np.convolve(ensemble_pred, [0.2, 0.6, 0.2], mode='same')
                                ensemble_pred = smoothed_pred
                            except:
                                # Skip smoothing if it fails
                                pass

                        # Final length alignment
                        final_pred_length = min(len(ensemble_pred), len(store_dept_val))
                        final_pred = ensemble_pred[:final_pred_length]
                        final_actual = store_dept_val['Weekly_Sales'].iloc[:final_pred_length].tolist()
                        final_holiday = store_dept_val['IsHoliday'].iloc[:final_pred_length].tolist()

                        predictions.extend(final_pred.tolist())
                        actuals.extend(final_actual)
                        holidays.extend(final_holiday)

                    except Exception as ensemble_error:
                        print(f"     ⚠️ Ensemble error for Store {store}, Dept {dept}: {ensemble_error}")
                        # Try simple averaging as fallback
                        try:
                            simple_avg = np.mean(weighted_predictions, axis=0)
                            predictions.extend(simple_avg[:len(store_dept_val)].tolist())
                            actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                            holidays.extend(store_dept_val['IsHoliday'].tolist())
                        except:
                            # Ultimate fallback to mean prediction
                            fallback_pred = val_data['Weekly_Sales'].mean()
                            predictions.extend([fallback_pred] * len(store_dept_val))
                            actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                            holidays.extend(store_dept_val['IsHoliday'].tolist())
                else:
                    # No valid predictions found - use fallback
                    fallback_pred = val_data['Weekly_Sales'].mean()
                    predictions.extend([fallback_pred] * len(store_dept_val))
                    actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                    holidays.extend(store_dept_val['IsHoliday'].tolist())

            except Exception as e:
                print(f"     ⚠️ Error processing Store {store}, Dept {dept}: {e}")
                # Fallback prediction for this Store-Dept combination
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ]
                if len(store_dept_val) > 0:
                    fallback_pred = val_data['Weekly_Sales'].mean()
                    predictions.extend([fallback_pred] * len(store_dept_val))
                    actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                    holidays.extend(store_dept_val['IsHoliday'].tolist())

    except Exception as e:
        print(f"❌ Advanced ensemble prediction failed: {e}")
        print("   🔄 Using fallback predictions...")

        # Fallback to mean prediction
        fallback_pred = val_data['Weekly_Sales'].mean()
        predictions = [fallback_pred] * len(val_data)
        actuals = val_data['Weekly_Sales'].tolist()
        holidays = val_data['IsHoliday'].tolist()

    print(f"✅ Advanced ensemble predictions complete!")
    print(f"   🎯 Total predictions: {len(predictions):,}")
    print(f"   📊 Model-configuration combinations used: {len(all_predictions)}")

    return np.array(predictions), np.array(actuals), np.array(holidays)


In [38]:
def prepare_timesfm_data(train_data, val_data, train_holidays, val_holidays):
    """Prepare data for TimesFM forecasting"""
    print("📊 Preparing TimesFM data...")

    # Combine train and validation for full time series
    full_data = pd.concat([train_data, val_data]).reset_index(drop=True)
    full_holidays = pd.concat([train_holidays, val_holidays]).reset_index(drop=True)

    # Get unique Store-Dept combinations from regular DataFrame columns
    train_combinations = set(zip(train_data['Store'], train_data['Dept']))
    val_combinations = set(zip(val_data['Store'], val_data['Dept']))

    print(f"   📈 Train combinations: {len(train_combinations):,}")
    print(f"   📉 Val combinations: {len(val_combinations):,}")
    print(f"   🔄 Common combinations: {len(train_combinations & val_combinations):,}")

    return full_data, full_holidays, train_combinations, val_combinations



In [39]:
def main():
    """Main experiment execution"""
    print("🚀 Starting Experiment 12: TimesFM (Time Series Foundation Model)")
    print("=" * 80)

    # Check TimesFM availability
    if not TIMESFM_AVAILABLE:
        print("❌ TimesFM is not available. Please install it first:")
        print("   pip install timesfm torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q")
        print("   # For CUDA support: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
        print("   # For advanced covariates support: pip install jax jaxlib")
        print("   # For enhanced ensemble features: pip install scipy")
        return

    # Setup MLflow tracking
    experiment_name = setup_mlflow()

    with mlflow.start_run(run_name="TimesFM_Foundation_Model") as run:
        print(f"🔄 Starting MLflow run: {run.info.run_id}")

        # Log experiment metadata
        mlflow.log_param("experiment_type", "TimesFM_Foundation_Model")
        mlflow.log_param("model_type", "TimesFM_Pretrained")
        mlflow.log_param("feature_engineering", "Experiment_7_Pipeline")
        mlflow.log_param("data_split", "temporal_80_20")
        mlflow.log_param("foundation_model", "google/timesfm-1.0-200m-pytorch")
        mlflow.log_param("approach", "Pretrained_Foundation_Model")

        try:
            # Step 1: Get preprocessed data
            print("\n📊 Step 1: Data preprocessing...")
            X_train, y_train, X_val, y_val, train_holidays, val_holidays, split_info, feature_columns, train_processed, val_processed = get_preprocessed_data()

            # Reconstruct train and val dataframes for TimesFM
            # We need to get the original processed data with Store and Dept columns
            train_data = train_processed[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].copy()
            val_data = val_processed[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].copy()

            # Log data info
            mlflow.log_metric("train_samples", len(train_data))
            mlflow.log_metric("val_samples", len(val_data))
            mlflow.log_metric("total_features", len(feature_columns))
            mlflow.log_param("split_date", str(split_info['split_date']))

            # Step 2: Prepare TimesFM data
            print("\n📊 Step 2: Preparing TimesFM data...")
            full_data, full_holidays, train_combinations, val_combinations = prepare_timesfm_data(
                train_data, val_data, train_holidays, val_holidays
            )

            # Log combination info
            mlflow.log_metric("train_combinations", len(train_combinations))
            mlflow.log_metric("val_combinations", len(val_combinations))
            mlflow.log_metric("common_combinations", len(train_combinations & val_combinations))

            # Step 3: Prepare advanced TimesFM data with configurations
            print("\n🤖 Step 3: Preparing advanced TimesFM configurations...")

            # OPTION: Limit models for faster testing (uncomment next line for testing)
            max_models_for_testing = 100  # Test with only 100 models for speed
            # max_models_for_testing = None   # Use None for full training

            time_series_configs, combination_info, covariates_data = prepare_advanced_timesfm_data(
                full_data, train_combinations, val_combinations, max_models=max_models_for_testing
            )

            # Log advanced data info
            mlflow.log_metric("successful_models", len(combination_info))
            mlflow.log_metric("processed_combinations", len(combination_info))
            mlflow.log_metric("time_series_configurations", len(time_series_configs))

            # Initialize TimesFM model(s)
            print("\n🚀 Step 4: Initializing TimesFM model(s)...")

            # OPTION: Choose between simple and advanced ensemble
            use_advanced_ensemble = True  # Enable advanced multi-model ensemble

            if use_advanced_ensemble:
                print("   🎯 Using advanced multi-model ensemble...")
                models = initialize_multiple_timesfm_models()
                mlflow.log_param("ensemble_type", "advanced_multi_model")
                mlflow.log_metric("model_configurations", len(models))
            else:
                print("   🎯 Using simple single-model ensemble...")
                tfm = initialize_timesfm_model()
                models = {'standard': tfm}
                mlflow.log_param("ensemble_type", "simple_single_model")

            # Step 5: Make predictions
            print("\n🔮 Step 5: Making ensemble predictions...")

            if use_advanced_ensemble:
                y_pred, y_true, is_holiday = make_advanced_ensemble_predictions(
                    models, time_series_configs, combination_info, covariates_data, full_data, val_data
                )
            else:
                y_pred, y_true, is_holiday = make_ensemble_timesfm_predictions(
                    models['standard'], time_series_configs, combination_info, covariates_data, full_data, val_data
                )

            # Step 6: Calculate metrics
            print("\n📊 Step 6: Calculating metrics...")

            if len(y_pred) > 0 and len(y_true) > 0:
                # Standard metrics
                mae = mean_absolute_error(y_true, y_pred)
                rmse = np.sqrt(mean_squared_error(y_true, y_pred))
                r2 = r2_score(y_true, y_pred)

                # WMAE (Competition metric)
                wmae = calculate_wmae(y_true, y_pred, is_holiday)

                # Holiday vs non-holiday breakdown
                holiday_mask = is_holiday == True
                non_holiday_mask = is_holiday == False

                holiday_mae = mean_absolute_error(y_true[holiday_mask], y_pred[holiday_mask]) if holiday_mask.sum() > 0 else 0
                non_holiday_mae = mean_absolute_error(y_true[non_holiday_mask], y_pred[non_holiday_mask]) if non_holiday_mask.sum() > 0 else 0

                # Log all metrics
                mlflow.log_metric("mae", mae)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2_score", r2)
                mlflow.log_metric("wmae", wmae)
                mlflow.log_metric("holiday_mae", holiday_mae)
                mlflow.log_metric("non_holiday_mae", non_holiday_mae)
                mlflow.log_metric("holiday_samples", holiday_mask.sum())
                mlflow.log_metric("non_holiday_samples", non_holiday_mask.sum())

                # Step 7: Results summary
                print("\n" + "=" * 60)
                print("🎯 EXPERIMENT 12 RESULTS SUMMARY (ADVANCED TIMESFM)")
                print("=" * 60)
                print(f"📊 Validation Metrics:")
                print(f"   WMAE (Competition Metric): ${wmae:,.2f}")
                print(f"   MAE: ${mae:,.2f}")
                print(f"   RMSE: ${rmse:,.2f}")
                print(f"   R²: {r2:.4f}")
                print(f"\n📊 Holiday Breakdown:")
                print(f"   Holiday MAE: ${holiday_mae:,.2f} ({holiday_mask.sum():,} samples)")
                print(f"   Non-Holiday MAE: ${non_holiday_mae:,.2f} ({non_holiday_mask.sum():,} samples)")
                print(f"\n📊 Advanced Model Statistics:")
                print(f"   Time series configurations: {len(time_series_configs)}")
                print(f"   Model configurations: {len(models) if use_advanced_ensemble else 1}")
                print(f"   Store-Dept combinations: {len(train_combinations):,}")
                print(f"   Ensemble type: {'Advanced Multi-Model' if use_advanced_ensemble else 'Simple Single-Model'}")
                print(f"   Foundation model: google/timesfm-1.0-200m-pytorch")
                print(f"   Covariates support: Enabled")
                print(f"   Data transformations: Raw, Log, Normalized, Detrended")
                print(f"   Post-processing: Weighted ensemble + smoothing")

                # Step 8: Save artifacts
                print(f"\n💾 Saving model artifacts...")

                # Save model summary
                model_summary = {
                    'experiment_name': experiment_name,
                    'run_id': run.info.run_id,
                    'time_series_configurations': len(time_series_configs),
                    'model_configurations': len(models) if use_advanced_ensemble else 1,
                    'ensemble_type': 'advanced_multi_model' if use_advanced_ensemble else 'simple_single_model',
                    'foundation_model': 'google/timesfm-1.0-200m-pytorch',
                    'advanced_features': {
                        'covariates_support': True,
                        'data_transformations': ['raw', 'log', 'normalized', 'detrended'],
                        'model_variants': list(models.keys()) if use_advanced_ensemble else ['standard'],
                        'weighted_ensemble': True,
                        'post_processing': 'smoothing'
                    },
                    'feature_columns': feature_columns,
                    'metrics': {
                        'wmae': wmae,
                        'mae': mae,
                        'rmse': rmse,
                        'r2': r2
                    }
                }

                with open('timesfm_exp12_summary.json', 'w') as f:
                    json.dump(model_summary, f, indent=2, default=str)

                mlflow.log_artifact('timesfm_exp12_summary.json')

                print("✅ Experiment 12 completed successfully!")

            else:
                print("❌ No predictions generated - check data and model setup")
                mlflow.log_param("error", "No predictions generated")

        except Exception as e:
            print(f"❌ Experiment failed: {e}")
            mlflow.log_param("error", str(e))
            raise

    print("\n🎉 Experiment 12: TimesFM Foundation Model - COMPLETE!")



In [40]:
if __name__ == "__main__":
    main()

🚀 Starting Experiment 12: TimesFM (Time Series Foundation Model)
🔧 Setting up MLflow tracking...


Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

   ✅ Using existing experiment: walmart_sales_forecasting_timesfm
   📊 MLflow experiment: walmart_sales_forecasting_timesfm
🔄 Starting MLflow run: 06726a66d8734cba800ceb7e29a4f0d9

📊 Step 1: Data preprocessing...
📊 Getting preprocessed data...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data
🔄 Transforming training data...
   🧹 Removed 45,193 outliers (13.4%)
   📝 Removed 5 markdown features
   🔄 Removed 3 redundant fe

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 3 TimesFM model configurations initialized:
   📊 standard: ready
   📊 long_context: ready
   📊 short_horizon: ready

🔮 Step 5: Making ensemble predictions...
🔮 Making advanced ensemble predictions...
   🔮 Running advanced ensemble with 3 models and 4 configurations...
   🤖 Using standard model...
     📊 Forecasting with standard_raw_sales...
       ⚠️ Covariates failed, using standard forecast for standard_raw_sales
       ✅ standard_raw_sales complete! Shape: (100, 64)
     📊 Forecasting with standard_log_sales...
       ✅ standard_log_sales complete! Shape: (100, 64)
     📊 Forecasting with standard_normalized_sales...
       ✅ standard_normalized_sales complete! Shape: (100, 64)
     📊 Forecasting with standard_detrended_sales...
       ✅ standard_detrended_sales complete! Shape: (100, 64)
   🤖 Using long_context model...
     📊 Forecasting with long_context_raw_sales...
       ✅ long_context_raw_sales complete! Shape: (100, 32)
     📊 Forecasting with long_context_log_sales...
  